In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [12]:
def conv1x1(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)

def conv3x3(in_channels, out_channels, stride=1, padding=1, dilation=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=padding, bias=False, dilation=dilation)

class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super().__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        
        if self.downsample is not None:
            out += self.downsample(x)
        else:
            out += x
        
        out = self.relu(out)

        return out

def make_layer(block, in_channels, out_channels, n_blokcs=2, stride=1):
    if stride != 1:
        downsample = nn.Sequential(conv1x1(in_channels, out_channels, stride))
    else:
        downsample = None
    
    layers = []
    layers.append(block(in_channels, out_channels, stride, downsample))
    for _ in range(1, n_blokcs):
        layers.append(block(out_channels, out_channels))
    return nn.Sequential(*layers)

In [13]:
class Resnet(nn.Module):
    def __init__(self, n_classes=10):
        super().__init__()
        self.n_classes = n_classes
        
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = make_layer(BasicBlock, 64, 64)
        self.layer2 = make_layer(BasicBlock, 64, 128, stride=2)
        self.layer3 = make_layer(BasicBlock, 128, 256, stride=2)
        self.layer4 = make_layer(BasicBlock, 256, 512, stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, n_classes)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

In [14]:
from torchsummary import summary

In [15]:
model = Resnet().to(device)

In [21]:
summary(model, (3, 32, 32), device=device.type)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]           9,408
       BatchNorm2d-2           [-1, 64, 16, 16]             128
              ReLU-3           [-1, 64, 16, 16]               0
         MaxPool2d-4             [-1, 64, 8, 8]               0
            Conv2d-5             [-1, 64, 8, 8]          36,864
       BatchNorm2d-6             [-1, 64, 8, 8]             128
              ReLU-7             [-1, 64, 8, 8]               0
            Conv2d-8             [-1, 64, 8, 8]          36,864
       BatchNorm2d-9             [-1, 64, 8, 8]             128
             ReLU-10             [-1, 64, 8, 8]               0
       BasicBlock-11             [-1, 64, 8, 8]               0
           Conv2d-12             [-1, 64, 8, 8]          36,864
      BatchNorm2d-13             [-1, 64, 8, 8]             128
             ReLU-14             [-1, 6

In [17]:
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=False, 
                                            transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100, 
                                          shuffle=False)

Files already downloaded and verified


In [18]:
num_epochs = 80
learning_rate = 0.001

In [19]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [20]:
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

total_step = len(train_loader)
curr_lr = learning_rate
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

Epoch [1/80], Step [100/500] Loss: 1.7437
Epoch [1/80], Step [200/500] Loss: 1.4807
Epoch [1/80], Step [300/500] Loss: 1.5566
Epoch [1/80], Step [400/500] Loss: 1.5227
Epoch [1/80], Step [500/500] Loss: 1.5672
Epoch [2/80], Step [100/500] Loss: 1.4165
Epoch [2/80], Step [200/500] Loss: 1.1463
Epoch [2/80], Step [300/500] Loss: 1.2914
Epoch [2/80], Step [400/500] Loss: 1.0318
Epoch [2/80], Step [500/500] Loss: 1.2380
Epoch [3/80], Step [100/500] Loss: 1.0928
Epoch [3/80], Step [200/500] Loss: 0.8519
Epoch [3/80], Step [300/500] Loss: 1.1353
Epoch [3/80], Step [400/500] Loss: 0.9011
Epoch [3/80], Step [500/500] Loss: 0.8858
Epoch [4/80], Step [100/500] Loss: 0.7551
Epoch [4/80], Step [200/500] Loss: 0.9546
Epoch [4/80], Step [300/500] Loss: 1.1054
Epoch [4/80], Step [400/500] Loss: 0.9131
Epoch [4/80], Step [500/500] Loss: 1.0048
Epoch [5/80], Step [100/500] Loss: 1.1182
Epoch [5/80], Step [200/500] Loss: 0.7873
Epoch [5/80], Step [300/500] Loss: 0.7456
Epoch [5/80], Step [400/500] Loss:

In [ ]:

# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))
